In [ ]:
library(lme4)
library(readr)

setwd("C:/Users/s1155063404/Desktop/Projects/brazilian-ecommerce-dataset/StatisticalAnalysis")

dataset = read_csv("./dataset.csv")

#Standardize the observations for modeling
standardize <- function(x){
  mean_x = mean(x)
  sig_x = sqrt(var(x))
  if (sig_x > 0){
    return((x - mean_x) / sig_x)
  }
  else {
    return(x)
  }
}

dataset$size = standardize(dataset$size)
dataset$order_products_value = standardize(dataset$order_products_value)
dataset$order_freight_value = standardize(dataset$order_freight_value)
dataset$distance = standardize(dataset$distance)

## Statistical Modelling:
Hypothesis: There is correlation for delivery time of the customers within same city

Proposed modeling: GLM (base model) vs GLMM (which assumes correlation for observations within city)

Remark: I do not use any features of the location of the customer in this modeling. I analyze the effect of location in analysis2.

In [2]:
#GLMM
fit = glmer(delivery_time ~ size + order_products_value + order_freight_value +
              distance + (1 + distance | cluster), 
            data = dataset, family=Gamma(link="log"))

summary(fit)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.00430913 (tol = 0.001, component 1)"

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: Gamma  ( log )
Formula: delivery_time ~ size + order_products_value + order_freight_value +  
    distance + (1 + distance | cluster)
   Data: dataset

      AIC       BIC    logLik  deviance  df.resid 
 608457.9  608543.1 -304220.0  608439.9     95190 

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-1.470 -0.591 -0.222  0.310 32.396 

Random effects:
 Groups   Name        Variance Std.Dev. Corr 
 cluster  (Intercept) 0.03026  0.1740        
          distance    0.01928  0.1389   -0.39
 Residual             0.42935  0.6552        
Number of obs: 95199, groups:  cluster, 4030

Fixed effects:
                     Estimate Std. Error t value Pr(>|z|)    
(Intercept)          2.583320   0.006171 418.649  < 2e-16 ***
size                 0.055366   0.002203  25.135  < 2e-16 ***
order_products_value 0.003849   0.001988   1.936   0.0528 .  
order_freight_value  0.017039   0.002557 

In [3]:
#GLM
base = glm(delivery_time ~ size + order_products_value + order_freight_value + distance,
           data = dataset, family=Gamma(link="log"))

summary(base)


Call:
glm(formula = delivery_time ~ size + order_products_value + order_freight_value + 
    distance, family = Gamma(link = "log"), data = dataset)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3975  -0.4888  -0.1689   0.1990   5.5254  

Coefficients:
                     Estimate Std. Error  t value Pr(>|t|)    
(Intercept)          2.505396   0.002270 1103.477   <2e-16 ***
size                 0.042392   0.002702   15.691   <2e-16 ***
order_products_value 0.001857   0.002498    0.744    0.457    
order_freight_value  0.041792   0.002994   13.959   <2e-16 ***
distance             0.262885   0.002452  107.212   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for Gamma family taken to be 0.4907483)

    Null deviance: 40696  on 95198  degrees of freedom
Residual deviance: 33320  on 95194  degrees of freedom
AIC: 619468

Number of Fisher Scoring iterations: 5


Findings: 
1. GLMM is statistically more appropriate based on AIC score, which support my Hypothesis.
2. The distance, size and the order_freight_value are positively related to the delivery time and are significant, while order_products_value is less significant.
3. The coefficient of distance is the largest in magnitude, which means the effect of distance on delivery time is the strongest.
